In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'pandas'

In [62]:
# Load the data
df = pd.read_csv('labeled-data.csv')

# Exclude examples labeled 'N' or 'Done'
df_filtered = df[(df['label'] != 'N') & (df['label'] != 'Done')]

# Extract features and labels
X = df_filtered.iloc[:, 2:9].values  # assuming your features start from the 3rd column
y = df_filtered['label'].values

In [63]:
# Specify the order of your classes
class_order = ['L', 'R']  # Adjust this based on your actual class order

# Initialize and fit the LabelEncoder with specified classes
label_encoder = LabelEncoder()
label_encoder.fit(class_order)

# Transform labels to encoded format
y_encoded = label_encoder.transform(y)

In [64]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Reshape the data for Conv1D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [65]:
# Assuming X_train.shape[1] is the number of features in your data
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())  # Flatten the output before the Dense layer
model.add(Dense(64, activation='relu'))  # Second layer is Dense with 64 neurons and ReLU activation
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


In [66]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [67]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6039 - accuracy: 0.6873 - val_loss: 0.5893 - val_accuracy: 0.6742
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5716 - accuracy: 0.6948 - val_loss: 0.5837 - val_accuracy: 0.6742
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5536 - accuracy: 0.7112 - val_loss: 0.5702 - val_accuracy: 0.6629
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5459 - accuracy: 0.7251 - val_loss: 0.5770 - val_accuracy: 0.6629
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5312 - accuracy: 0.7352 - val_loss: 0.5830 - val_accuracy: 0.6854
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5220 - accuracy: 0.7364 - val_loss: 0.5807 - val_accuracy: 0.6854
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5191 - accuracy: 0.7352 - val_loss: 0.6022 - val_accuracy: 0.6854
Epoch 8/10
25/25 [==

In [68]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 1ms/step
Test Accuracy: 0.7346938775510204


In [2]:
%pip install pylsl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pylsl

# Find the stream on the lab network
print("Looking for an EEG stream...")
streams = pylsl.resolve_stream('type', 'EEG')

# Create a new inlet to read from the stream
inlet = pylsl.StreamInlet(streams[0])

while True:
    print("hello")
    # Get a new sample
    sample, timestamp = inlet.pull_sample()
    # Process the sample (e.g., feature extraction, scaling)
    processed_sample = preprocess(sample)
    
    # Predict using your model
    prediction = your_model.predict(processed_sample)
    
    # Process the prediction as needed
    handle_prediction(prediction)


Looking for an EEG stream...


2023-12-04 08:06:34.310 (   0.001s) [          2B6F05]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2023-12-04 08:06:34.310 (   0.001s) [          2B6F05]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2023-12-04 08:06:34.310 (   0.001s) [          2B6F05]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2023-12-04 08:06:34.310 (   0.001s) [          2B6F05]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2023-12-04 08:06:34.310 (   0.001s) [          2B6F05]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2023-12-04 08:06:34.311 (   0.001s) [          2B6F05]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2023-12-04 08:06:34.311 (   0.001s) [          2B6F05]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2023-12-04 08:06:34.311 (   0.001s) [          2B6F05]      netinterfaces.cpp:91    I